In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import string
import re

In [3]:
df = pd.read_excel('../data/job_en.xlsx')
print(df.info())
print(df.describe())
print(df.isnull().sum())


  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company_name    3040 non-null   object 
 1   city_text       3044 non-null   object 
 2   exp             3047 non-null   float64
 3   item_tag        3046 non-null   object 
 4   company_scale   2249 non-null   object 
 5   company_field   3047 non-null   object 
 6   position        3047 non-null   object 
 7   education       2735 non-null   object 
 8   min_salary      3047 non-null   float64
 9   max_salary      3047 non-null   float64
 10  average_salary  3047 non-null   float64
 11  platform        3047 non-null   object 
 12  job_des         3046 non-null   object 
 13  job_require     3047 non-null   object 
dtypes: float64(4), object(10)
memory usage: 333.4+ KB
None
               exp    min_salary    max_salary  average_salary
count  3047.000000  3.047000e+03  3.047000e+03    

In [4]:
skill_df = pd.read_csv('..\data\skill.csv') 


In [ ]:

skill_to_group = dict(zip(skill_df['skill'], skill_df['group_skill']))
skills = list(skill_to_group.keys())

# Extract_skills và group function
def extract_skills_and_groups(row):
    text_combined = f"{row['job_des']} {row['job_require']}".lower()
    matched_skills = set()
    matched_groups = set()
    
    for skill in skills:
        pattern = r'(?<!\w)' + re.escape(skill) + r'(?!\w)'
        if re.search(pattern, text_combined):
            matched_skills.add(skill.strip().lower())
            matched_groups.add(skill_to_group[skill].strip())

    return pd.Series({
        'skills': ', '.join(sorted(matched_skills)),
        'group_skill': ', '.join(sorted(matched_groups))
    })


# Apply vào dataframe
test_df = df.head(5).copy()
test_df[['skills', 'group_skill']] = test_df.apply(extract_skills_and_groups, axis=1)

print(test_df[['job_des', 'job_require', 'skills', 'group_skill']])

                                             job_des  \
0  Mô tả công việc Navigos Search's Client is loo...   
1  Mô tả công việc What You'll Be Doing • Lead ar...   
2  Mô tả công việc What You'll Be Doing • Lead ar...   
3  Mô tả công việcTOP 4 JOB RESPONSIBILITIES Qual...   
4  Mô tả Công việc         - Work Location: Phu N...   

                                         job_require  \
0  Yêu cầu công việc - A Bachelor’s degree in a r...   
1  Yêu cầu công việc • 6+ years of full-stack dev...   
2  Yêu cầu công việc • 6+ years of full-stack dev...   
3  Yêu cầu công việcRequirements & Experience • 3...   
4  YÊU CẦU CÔNG VIỆC <Necessary Skill / Experienc...   

                                              skills  \
0  agile, blockchain, data, go, performance, qa, ...   
1  api, architecture, design, docker, graphql, pe...   
2  api, architecture, design, docker, graphql, pe...   
3                                                      
4  agile, architecture, design, docker, expres

In [6]:
#apply for all df
df[['skills', 'group_skill']] = df.apply(extract_skills_and_groups, axis=1)

In [7]:
# for all cells in skills, if it is empty, print job_des
for index, row in test_df.iterrows():
    if not row['skills']:
        print(f"Empty skills for job_des: {row['job_des']}")

Empty skills for job_des: Mô tả công việcTOP 4 JOB RESPONSIBILITIES Quality Assurance and Detail Orientation • Perform rigorous quality checks on all images before delivery, ensuring they meet our high standards and the specific requirements of each customer. • Attention to detail is crucial, as this role demands the ability to spot minor discrepancies or errors and correct them promptly. Understanding Customer Aesthetics • Have a keen eye for fashion aesthetics and the ability to understand and adapt to various customer preferences and styles. • Serve as the bridge between our production team and customers, ensuring that each image not only meets technical standards but also aligns with the customer's vision and brand identity. Proactiveness in Operational Improvement • The role requires a proactive approach to identifying opportunities for improvement within the production process. • This includes suggesting enhancements to workflows, technology, and techniques based on customer feed

In [8]:
# count each skill have many times in df
skills_count = df['skills'].str.split(', ').explode().value_counts()
# print
print("Skills count:")
print(skills_count)

Skills count:
skills
design          1530
data             768
performance      593
testing          479
web              393
                ... 
wcf                1
unity3d            1
voip               1
xamarin            1
unit-testing       1
Name: count, Length: 203, dtype: int64


In [9]:
# count each skill have many times in df
group_count = df['group_skill'].str.split(', ').explode().value_counts()
# print
print("Skills count:")
print(group_count)

Skills count:
group_skill
Process & Methods      2160
OS & Infrastructure    1262
Data Systems           1148
Languages              1030
Libs & Frameworks       679
Development Tools       638
                        325
Name: count, dtype: int64


In [10]:
# save to csv
output_path = '../data/job_en_with_skills.csv'

In [11]:
# print path saved
print(f"Saving to {output_path}")
df.to_csv(output_path, index=False)


Saving to ../data/job_en_with_skills.csv
